In [ ]:
!pip install pytorch-transformers
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers
!git clone https://github.com/e9t/nsmc.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=99aabacc17814b2d7dbee012763236b52d0c05abad0468603bac7e1685c83cfe
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-3gve5mp4/kobert-tokenizer_10a25fc614ec4f3388337ba9d9dc253c
  Running command git clone --filter=blob:none --qu

In [ ]:
import numpy as np
import pandas as pd
import re
import torch
from transformers import AutoTokenizer,AutoModelForSequenceClassification,pipeline
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
import urllib.request
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
from transformers import TFBertForSequenceClassification
import tensorflow as tf
import os
from tqdm.auto import tqdm
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel,BertForTokenClassification
import sentencepiece as spm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import torch.nn as nn

In [ ]:
# 토크나이저 지정
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert",return_tensors='pt')

In [ ]:

# 평가할 금융분석 데이터 불러오기
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename='finance_data.csv')

('finance_data.csv', <http.client.HTTPMessage at 0x7cbcc8c6c070>)

In [ ]:
# 평가할 금융분석 데이터 불러오기 => 5천개의 데이터셋
data = pd.read_csv('finance_data.csv')
del data['sentence']

# 0: 중립
# 1: 긍정
# 2: 부정
data['labels'] = data['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])

# 중복 제거
duplicate = data[data.duplicated()]
data.drop_duplicates(subset=['kor_sentence'], inplace=True)
print('총 샘플의 수 :',len(data))

# 중립 제거
data=data[data['labels']!=0]
data.shape

총 샘플의 수 : 4827


(1966, 2)

In [ ]:
# 중복 제거
duplicate = data[data.duplicated()]
data.drop_duplicates(subset=['kor_sentence'], inplace=True)
print('총 샘플의 수 :',len(data))

총 샘플의 수 : 1966


In [ ]:
# 중립 제거
data=data[data['labels']!=0]
data

,labels,kor_sentence
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
5,1,ASPOCOMP의 성장기에 대한 자금 조달은 기술적으로 더 까다로운 HDI 인쇄 회...
6,1,2010년 4분기 Componenta의 순매출은 전년 동기의 7600만 유로에서 2...
...,...,...
4840,2,헬싱키 톰슨 파이낸셜 - 카고텍의 주가는 화물 취급 그룹이 4월부터 6월까지 3개월...
4841,2,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4843,2,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,2,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [ ]:
# 텍스트 데이터와 label 데이터 지정
data.reset_index(inplace=True,drop=True)
data_kr=data['kor_sentence']
data_label=data['labels']

In [ ]:
from google.colab import drive

# Mydrive 사용
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,pipeline

# models = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert")

models=AutoModelForSequenceClassification.from_pretrained("./drive/MyDrive/model_directory/") # => KR-FinBert에서 weight 찾아서 저장후 불러온 모델

classifier=pipeline("sentiment-analysis",tokenizer=tokenizer,model=models,device=0) # pipeline을 사용하여 tokenizer와 model 연결후 감성분석

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBert and are newly initialized: ['classifier.weight', 'classifier.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 저장한 모델 신경망 파라미터들
# {
#   "_name_or_path": "snunlp/KR-FinBert",
#   "architectures": [
#     "BertForSequenceClassification"
#   ],
#   "attention_probs_dropout_prob": 0.1,
#   "classifier_dropout": null,
#   "gradient_checkpointing": false,
#   "hidden_act": "gelu",
#   "hidden_dropout_prob": 0.1,
#   "hidden_size": 768,
#   "initializer_range": 0.02,
#   "intermediate_size": 3072,
#   "layer_norm_eps": 1e-12,
#   "max_position_embeddings": 512,
#   "model_type": "bert",
#   "num_attention_heads": 12,
#   "num_hidden_layers": 12,
#   "pad_token_id": 0,
#   "position_embedding_type": "absolute",
#   "torch_dtype": "float32",
#   "transformers_version": "4.30.2",
#   "type_vocab_size": 2,
#   "use_cache": true,
#   "vocab_size": 20000
# }


In [ ]:
# models.save_pretrained('./model_directory') => 모델 신경망 저장

In [ ]:
# results => label과 score가 담김
# label_ => label만 추출
results=[]
label_=[]

for text in data_kr:
    result = classifier(text)
    results.append(result)

# label_에 긍,부정과 그에 대한 scrores 확률이 담김
for i, result in enumerate(results):
    for r in result:
        label_.append(r['label']) # label로 긍정과 부정만 추출
data['predict']=label_ # predict 컬럼에 담기

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
data['predict'].value_counts(),data['labels'].value_counts() # 긍부정 label 비교

(LABEL_1    1255
 LABEL_0     711
 Name: predict, dtype: int64,
 1    1362
 2     604
 Name: labels, dtype: int64)

In [ ]:
data['predict']=data['predict'].map({'LABEL_0':1,'LABEL_1':2})  # LABEL_0은 긍정, LABEL_1은 부정이므로 1,2로 긍정,부정을 나눔

In [ ]:
models = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert")

classifier=pipeline("sentiment-analysis",tokenizer=tokenizer,model=models,device=0) # pipeline을 사용하여 tokenizer와 model 연결후 감성분석

results=[]
label_=[]

for text in data_kr:
    result = classifier(text)
    results.append(result)

# label_에 긍,부정과 그에 대한 scrores 확률이 담김
for i, result in enumerate(results):
    for r in result:
        label_.append(r['label']) # label로 긍정과 부정만 추출
data['predict']=label_ # predict 컬럼에 담기

data['predict']=data['predict'].map({'LABEL_0':1,'LABEL_1':2})

TP = 0
TN = 0
FP = 0
FN = 0

dl = data['labels']
dp = data['predict']

for idx, da in enumerate(dl):
    if (dp[idx] == 1) and (dl[idx] == 1):
        TP += 1
    elif (dp[idx] == 2) and (dl[idx] == 2):
        TN += 1
    elif (dp[idx] == 1) and (dl[idx] == 2):
        FP += 1
    elif (dp[idx] == 2) and (dl[idx] == 1):
        FN += 1

precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

print('정밀도(Precision): ', precision)
print('재현율(Recall): ', recall)
print('F1 score: ', f1_score)
print('정확도(Accuracy): ', accuracy)


정밀도(Precision):  0.7010036978341257
재현율(Recall):  0.9743024963289281
F1 score:  0.8153609831029186
정확도(Accuracy):  0.7243031536113937


In [ ]:
from google.colab import drive

# Mydrive 사용
drive.mount('/content/drive')

# label있는 신세계 데이터 불러오기
shin=pd.read_csv('/content/drive/MyDrive/shin_result.csv',header=0,index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
shin=shin[shin['sentiment']!=0]
shin=shin.reset_index(drop=True)

In [ ]:
# label있는 신세계 데이터 불러오기
# 3000개의 데이터셋
shin=pd.read_csv('/content/drive/MyDrive/shin_result.csv',header=0,index_col=0)
shin=shin[shin['sentiment']!=0]
shin=shin.reset_index(drop=True)
shin['sentiment'].value_counts() # 2362개 긍정 , 568개 부정

1    2362
2     486
Name: sentiment, dtype: int64

In [ ]:
shin.drop_duplicates(subset=['title'], inplace=True) #중복제거

In [ ]:

shin_kr=shin['title']
shin_lis=[]
for text in shin_kr:
    result = classifier(text)
    shin_lis.append(result)

shin_li=[]
for i, result in enumerate(shin_lis):
    for r in result:
        shin_li.append(r['label'])
shin['predict']=shin_li
shin['predict']=shin['predict'].map({'LABEL_1':2,'LABEL_0':1})


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
shin['predict'].value_counts(),shin['sentiment'].value_counts()

(1    2388
 2     460
 Name: predict, dtype: int64,
 1    2362
 2     486
 Name: sentiment, dtype: int64)

In [ ]:
shin['predict']=shin['predict'].map({'LABEL_1':2,'LABEL_0':1})

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

sent=shin['sentiment']
pred=shin['predict']
for idx, da in enumerate(sent):
    if (pred[idx] == 1) and (sent[idx] == 1):
        TP += 1
    elif (pred[idx] == 2) and (sent[idx] == 2):
        TN += 1
    elif (pred[idx] == 1) and (sent[idx] == 2):
        FP += 1
    elif (pred[idx] == 2) and (sent[idx] == 1):
        FN += 1

precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

print('정밀도(Precision): ', precision)
print('재현율(Recall): ', recall)
print('F1 score: ', f1_score)
print('정확도(Accuracy): ', accuracy)


정밀도(Precision):  0.8189245887159103
재현율(Recall):  0.8779592803030303
F1 score:  0.8474150242787775
정확도(Accuracy):  0.7958610846812559


In [ ]:
shin

,date,title,sentiment,predict
0,2023-04-18 16:05,"[현장] 현대백화점 디즈니스토어 문전성시, 정지선 오프라인 '경험'을 판다",1,1
1,2023-04-13 20:01,현대백화점-디즈니 손잡았다…올해 안에 '디즈니스토어' 네 곳 연다,1,1
2,2023-04-07 13:56,"현대백화점, 한국관광공사 맞손…글로벌 MZ세대 공략 나선다",1,1
3,2023-04-07 17:25,"현대백화점, 관광공사와 손잡고 글로벌 MZ세대 잡는다",1,1
4,2023-03-07 07:53,"현대백화점, 新성장동력 '비건뷰티' 키운다",1,1
...,...,...,...,...
3583,2023-03-24 09:25,"""그래도 믿을 건 중국 뿐?""…부진한 의류株, 돌파구 있나",1,1
3584,2023-05-23 17:49,"""그동안 몰라봐서 미안""…매출 47% 뛰게 만든 '이것'의 힘",1,1
3585,2023-02-21 08:53,"""고물가에 '벌크업 쇼핑' 뜬다""…G마켓, 대용량제품 매출 3배↑",1,1
3586,2023-02-21 11:56,"""고물가 시대""…G마켓, 대용량 제품 매출 3배 증가",1,1
